## Flask-Socketio
extension for Flask that enables **real-time communication** between clients and the server using WebSockets. It allows you to build applications that can push updates to clients, which is useful for chat applications, notifications, and live updates.

In [ ]:
!pip install flask-socketio

In [ ]:
from flask import Flask,render_template
from flask_socketio import SocketIO

app = Flask(__name__)
# Initialize SocketIO with the Flask app
socketio = SocketIO(app)

### Creating SocketIO Events
Now, let’s add a SocketIO event to respond to a message sent from the client.

In [ ]:
# Handle the 'message' event from the client
@socketio.on('message')
def handle_message(msg):
    # Print the received message to the console
    print('Received message: ' + msg)
    # Send a response back to the client
    socketio.send('Response from server: ' + msg)

#### Example HTML File

In [ ]:
<script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/4.0.0/socket.io.js"></script>
    <script type="text/javascript">
        // Establish a connection to the Socket.IO server
        var socket = io();

        // Event handler for successful connection
        socket.on('connect', function() {
            console.log('Connected to server'); // Log to console
        });

        // Event handler for receiving messages from the server
        socket.on('message', function(msg) {
            console.log('Received message: ' + msg); // Log the received message
        });

        // Function to send a message to the server
        function sendMessage() {
            // Get the value from the input field
            var msg = document.getElementById('message').value;
            // Send the message to the server
            socket.send(msg);
            // Clear the input field
            document.getElementById('message').value = '';
        }
    </script>
<body>
    <h1>Flask-SocketIO Chat</h1>
    <!-- Input field for the user to type messages -->
    <input type="text" id="message" placeholder="Type a message" />
    <!-- Button to send the message -->
    <button onclick="sendMessage()">Send</button>
</body>

### Namespaces
allow you to separate different communication channels in your application. They provide a way to segment your WebSocket connections and handle events related to different functionalities independently.

#### Example of Namespaces
you might have a chat application with different channels for private messages, public chat rooms, and notifications. Each of these channels can be a different namespace.

In [ ]:
from flask import Flask
from flask_socketio import SocketIO, Namespace

app = Flask(__name__)
socketio = SocketIO(app)

# Create a custom namespace for chat functionality
class ChatNamespace(Namespace):
    def on_connect(self):
        print('Client connected to chat namespace.')

    def on_disconnect(self):
        print('Client disconnected from chat namespace.')

    def on_message(self, message):
        print(f'Received message: {message}')
        # Echo the message back to the client
        self.send(message)

# Register the namespace with the SocketIO server
socketio.on_namespace(ChatNamespace('/chat'))

if __name__ == '__main__':
    socketio.run(app)


##### HTML Client Code

In [ ]:
<!DOCTYPE html>
<html>
<head>
    <title>Flask-SocketIO with Namespaces</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/4.0.0/socket.io.js"></script>
    <script type="text/javascript">
        // Connect to the chat namespace
        var socket = io('/chat');

        // Event handler for receiving messages from the server
        socket.on('message', function(msg) {
            console.log('Received message: ' + msg);
        });

        // Function to send a message to the server
        function sendMessage() {
            var msg = document.getElementById('message').value;
            socket.send(msg); // Send the message to the server
            document.getElementById('message').value = ''; // Clear the input field
        }
    </script>
</head>
<body>
    <h1>Chat Namespace Example</h1>
    <input type="text" id="message" placeholder="Type a message" />
    <button onclick="sendMessage()">Send</button>
</body>
</html>


In this example, when the client connects to the /chat namespace, the server prints a message indicating the connection. Messages sent through this namespace will be handled independently from any other namespaces you might create.

### Rooms
allow you to group multiple sockets together. You can think of rooms as virtual spaces where multiple clients can join and communicate with each other.

#### Example of Rooms
Let’s create an example where clients can join different chat rooms.

In [ ]:
from flask import Flask, render_template
from flask_socketio import SocketIO, join_room, leave_room

app = Flask(__name__)
socketio = SocketIO(app)

@app.route('/')
def index():
    return render_template('index.html')  # Render the index page

# Handle the 'join' event to join a room
@socketio.on('join')
def on_join(data):
    room = data['room']  # Get the room name from the received data
    join_room(room)  # Join the specified room
    # Send a message to the room indicating the user has joined
    socketio.send(room, f'User has joined the room: {room}')

# Handle the 'message' event to send messages to a specific room
@socketio.on('message')
def handle_message(data):
    room = data['room']  # Get the room name from the received data
    msg = data['message']  # Get the message from the received data
    # Send the message to all clients in the specified room
    socketio.send(room, msg)

# Handle the 'leave' event to leave a room
@socketio.on('leave')
def on_leave(data):
    room = data['room']
    leave_room(room)  # Leave the specified room
    # Send a message to the room indicating the user has left
    socketio.send(room, f'User has left the room: {room}')

if __name__ == '__main__':
    socketio.run(app)


### Braodcasting Messages
You can broadcast messages to all clients in a specific room or all connected clients.

In [ ]:
@socketio.on('broadcast_message')
def handle_broadcast(msg):
    # Emit the message to all connected clients
    socketio.emit('message', msg, broadcast=True)

### Handling Disconnections
You can handle disconnections using the **on_disconnect** event.

In [ ]:
# Handle disconnection events
@socketio.on('disconnect')
def handle_disconnect():
    # Print a message when a client disconnects
    print('Client disconnected')

##### HTML Client Code

In [ ]:
<!DOCTYPE html>
<html>
<head>
    <title>Flask-SocketIO Rooms Example</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/4.0.0/socket.io.js"></script>
    <script type="text/javascript">
        var socket = io(); // Connect to the default namespace

        // Function to join a room
        function joinRoom() {
            var room = document.getElementById('room').value; // Get the room name
            socket.emit('join', { room: room }); // Emit a join event
        }

        // Event handler for receiving messages from the server
        socket.on('message', function(msg) {
            console.log('Received message: ' + msg); // Log the message
        });

        // Function to send a message to the room
        function sendMessage() {
            var room = document.getElementById('room').value; // Get the room name
            var msg = document.getElementById('message').value; // Get the message
            socket.send({ room: room, message: msg }); // Send message to the room
            document.getElementById('message').value = ''; // Clear the input field
        }

        // Function to leave a room
        function leaveRoom() {
            var room = document.getElementById('room').value; // Get the room name
            socket.emit('leave', { room: room }); // Emit a leave event
        }
    </script>
</head>
<body>
    <h1>Chat Room Example</h1>
    <input type="text" id="room" placeholder="Room Name" />
    <button onclick="joinRoom()">Join Room</button>
    <button onclick="leaveRoom()">Leave Room</button><br><br>
    <input type="text" id="message" placeholder="Type a message" />
    <button onclick="sendMessage()">Send</button>
</body>
</html>
